Оценки могут принимать значение [0, 1], т.е. задание, которое сделали асессоры, имеет бинарную шкалу.

Используя данные об оценках, установите, какие асессоры хуже всего справились с заданием. На какие показатели вы ориентировались и какие метрики вы использовали для ответа на этот вопрос? Можно ли предложить какие-то новые метрики для подсчета качества асессоров с учетом природы оценок у этого бинарного задания?


In [1]:
#импорт библиотек
import numpy as np 
import pandas as pd
from datetime import datetime

In [2]:
#считываем файл и сохраняем в переменную data
path = 'task2.csv'# путь к папке на локальном компьютере
data = pd.read_csv(path, sep='\t')
data

,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0
...,...,...,...,...,...
249995,assessor208,208,49999,1,1
249996,assessor139,139,49999,1,1
249997,assessor333,333,49999,1,1
249998,assessor160,160,49999,1,1


In [3]:
#Общая информация о данных таблицы data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   login   250000 non-null  object
 1   uid     250000 non-null  int64 
 2   docid   250000 non-null  int64 
 3   jud     250000 non-null  int64 
 4   cjud    250000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 9.5+ MB


Рассмотрим полученную информацию подробнее.

Пропущенных значений нет.

Всего в таблице 5 столбцов, тип данных у большинства - int.

Подробно разберём, какие в data столбцы и какую информацию они содержат:

login — логин асессора; 

uid — id асессора (user id); 

docid — id оцениваемого документа (document id); 

jud — оценка асессора (judgement); 

cjud — правильная оценка (correct judgement)

In [4]:
#Посмотрим, есть ли одинаковые задания у разных асессоров
len(data['docid'].unique())

50000

In [5]:
#Найдем количество асессоров, представленных в данном файле
len(data['uid'].unique())

600

In [6]:
docid_counts = data.docid.value_counts()
docid_counts.head(10)

2047     5
29860    5
17698    5
23841    5
21792    5
44223    5
42174    5
48317    5
46268    5
36027    5
Name: docid, dtype: int64

In [7]:
sum(data.docid.value_counts() == 5)

50000

Как мы видим, каждое задание оценивалось ровно 5 раз.

Добавим столбец с разницей двух столбцов с правильной оценкой и оценкой асессора. 

Если полученное значение = 0, то асессор правильно оценил задание, если 1 - то ошибся.

In [8]:
data['metrica1'] = abs(data['cjud'] - data['jud'])
data

,login,uid,docid,jud,cjud,metrica1
0,assessor158,158,0,0,0,0
1,assessor238,238,0,0,0,0
2,assessor488,488,0,0,0,0
3,assessor136,136,0,0,0,0
4,assessor300,300,0,0,0,0
...,...,...,...,...,...,...
249995,assessor208,208,49999,1,1,0
249996,assessor139,139,49999,1,1,0
249997,assessor333,333,49999,1,1,0
249998,assessor160,160,49999,1,1,0


Найдем, какое количество ошибок было допущено асессорами в данном файле.

In [9]:
sum(data.metrica1 > 0)

39678

In [10]:
data_pivot = data.pivot_table(columns = 'uid', values = 'metrica1', aggfunc=np.sum)
data_pivot = data_pivot.transpose()
data_pivot.head(10)

,metrica1
uid,
0,65
1,82
2,76
3,230
4,72
5,40
6,69
7,48
8,45


Чем меньше получившаяся сумма, тем лучше справился с работой асессор, так как допустил меньше ошибок.

Исходя из условий задания, необходимо установить, кто из асессоров хуже всего справился с заданиями, то есть сделал наибольшее количество ошибок.

In [11]:
data_pivot[data_pivot['metrica1'] == data_pivot['metrica1'].max()]

,metrica1
uid,
56,236


Наибольшее количество ошибок согласно metrica1 сделал асессор с uid = 56

Выведем топ-10 асессоров с наибольшим числом ошибок:

In [12]:
data_pivot_sorted = pd.DataFrame(np.sort(data_pivot.values, axis=0)[::-1][:10], columns=data_pivot.columns)
data_pivot_sorted

,metrica1
0,236
1,230
2,214
3,205
4,110
5,108
6,108
7,105
8,100
9,99


In [13]:
#Выведем uid асессоров и суммарное количество их ошибок
data_filter = data_pivot['metrica1'].isin(data_pivot_sorted['metrica1']) 
data_pivot[data_filter]

,metrica1
uid,
3,230
9,110
56,236
71,105
92,99
118,205
140,108
154,99
235,99


Давайте теперь попробуем оценить качество выполнения работ по другой метрике

In [14]:
#Посчитаем количество заданий, выполненное каждым пользователем
data_pivot_uid = data.pivot_table(columns = 'uid', values = 'docid', aggfunc='count')
data_pivot_uid = data_pivot_uid.transpose()
data_pivot_uid.head(10)

,docid
uid,
0,401
1,412
2,379
3,426
4,418
5,403
6,439
7,398
8,400


Найдем среднее число ошибок на 1 задание для каждого пользователя:

In [15]:
#Объединим две таблицы по значению uid
res = data_pivot.merge(data_pivot_uid, on=["uid"])
res.head(10)

,metrica1,docid
uid,,
0,65,401
1,82,412
2,76,379
3,230,426
4,72,418
5,40,403
6,69,439
7,48,398
8,45,400


In [16]:
res['metrica2'] = res['metrica1'] / res['docid']
res.head(10)

,metrica1,docid,metrica2
uid,,,
0,65,401,0.162095
1,82,412,0.199029
2,76,379,0.200528
3,230,426,0.539906
4,72,418,0.172249
5,40,403,0.099256
6,69,439,0.157175
7,48,398,0.120603
8,45,400,0.112500


Соответственно, чем это число выше, тем хуже асессор справляется с заданиями. 

Найдем асессора с максимальным значением metrica2

In [17]:
res[res['metrica2'] == res['metrica2'].max()]

,metrica1,docid,metrica2
uid,,,
56,236,411,0.574209


Это снова оказался тот же самый пользователь.

In [18]:
res_pivot_sorted = pd.DataFrame(np.sort(res.values, axis=0)[::-1][:10], columns=res.columns)
res_pivot_sorted

,metrica1,docid,metrica2
0,236.0,484.0,0.574209
1,230.0,481.0,0.539906
2,214.0,480.0,0.524297
3,205.0,467.0,0.519417
4,110.0,467.0,0.515152
5,108.0,467.0,0.273438
6,108.0,466.0,0.260047
7,105.0,461.0,0.257757
8,100.0,460.0,0.244444
9,99.0,460.0,0.242494


In [19]:
#Выведем uid асессоров и соотношение количества ошибок к количеству заданий
res_filter = res['metrica2'].isin(res_pivot_sorted['metrica2']) 
res[res_filter]

,metrica1,docid,metrica2
uid,,,
3,230,426,0.539906
9,110,423,0.260047
56,236,411,0.574209
71,105,433,0.242494
118,205,391,0.524297
154,99,405,0.244444
234,51,99,0.515152
335,108,419,0.257757
390,214,412,0.519417


Здесь топ-10 асессоров с худшим качеством выполнения заданий тоже не изменился

Т.к. исходная задача - задача бинарной классификации, мы можем попробовать построить confusion matrix и посчитать метрики для всех асесоров. 

Создадим столбец со следующими результатами:

TP - True Positive - столбцы jud=cjud=1

FN - False Negative - jud=0 cjud=1

FP - False Positive - jud=1 cjud=0

TN - True Negative - jud=0 chud=0

In [20]:
data.loc[(data.jud==data.cjud) & (data.jud==1), 'pred'] = 'TP'
data.loc[(data.jud==data.cjud) & (data.jud==0), 'pred'] = 'TN'
data.loc[(data.jud!=data.cjud) & (data.jud==0), 'pred'] = 'FN'
data.loc[(data.jud!=data.cjud) & (data.jud==1), 'pred'] = 'FP'

In [21]:
#Группируем данные по асессорам
assessors = data.groupby(['login', 'pred'])['pred'].count().unstack()
assessors

pred,FN,FP,TN,TP
login,,,,
assessor0,6,59,287,49
assessor1,4,78,289,41
assessor10,13,79,321,30
assessor100,15,68,299,35
assessor101,9,41,320,33
...,...,...,...,...
assessor95,10,29,349,40
assessor96,19,69,324,39
assessor97,7,33,346,37


In [22]:
#создадим основные метрики accuracy, precision, recall и f1
assessors['accuracy'] = (assessors['TP'] + assessors['TN']) / (assessors['TP'] + assessors['TN'] + assessors['FP'] + assessors['FN'])
assessors['recall'] = assessors['TP'] / (assessors['TP'] + assessors['FN'])
assessors['specifity'] = assessors['TN'] / (assessors['TN'] + assessors['FP'])
assessors['precision'] = assessors['TP'] / (assessors['TP'] + assessors['FP'])
assessors['f1_score'] = 2 * assessors['precision'] * assessors['recall'] / (assessors['precision'] + assessors['recall'])

In [23]:
assessors.f1_score.sort_values(ascending=False).tail(10)

login
assessor460    0.347826
assessor335    0.341463
assessor354    0.333333
assessor111    0.283186
assessor234    0.238806
assessor390    0.207407
assessor118    0.189723
assessor3      0.178571
assessor56     0.163121
assessor163    0.103896
Name: f1_score, dtype: float64

В данном списке, помимо выделенных ранее, появились новые асессоры. 